In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import chromadb

In [3]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2", device='cpu')

2025-04-08 23:26:43.001025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744126003.250145  275702 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744126003.320831  275702 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744126003.822303  275702 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744126003.822375  275702 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744126003.822382  275702 computation_placer.cc:177] computation placer alr

In [4]:
texts = ["Apple is a fruit", "Cars are vehicles", "I love mangoes"]

embeddings = embedder.encode(texts).tolist()

In [5]:
client = chromadb.PersistentClient(path="./test-chroma")

In [6]:
collection = client.get_or_create_collection("test-data")

# Store data
collection.add(
    embeddings=embeddings,
    documents=texts,
    metadatas=[{"topic": "fruit"}, {"topic": "vehicle"}, {"topic": "fruit"}],
    ids=["doc1", "doc2", "doc3"]
)

In [7]:
def query(text):
    query_embedding = embedder.encode(text).tolist()
    return collection.query(
        query_embeddings=query_embedding,
        n_results=2 # how many results to return
    )


print(query("This is a query document about fruit"))

{'ids': [['doc1', 'doc3']], 'embeddings': None, 'documents': [['Apple is a fruit', 'I love mangoes']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'topic': 'fruit'}, {'topic': 'fruit'}]], 'distances': [[0.7356013059616089, 1.1305949687957764]]}


In [9]:
import ollama

def chat(user_input):
    messages = []
    messages.append({'role': 'system', 'content': "You are a helpful AI assistent that answer user's questions"})
    messages.append({'role': 'user', 'content': user_input})
    
    response = ollama.chat(model='gemma3', messages=messages)
    reply = response['message']['content']
    return reply

user_message = 'Hello! how are you?'
response = chat(user_message)
print(response)

Hello there! I’m doing great, thank you for asking! As an AI, I don’t really *feel* in the way humans do, but my systems are running smoothly and I’m ready to help you with whatever you need. 😊

How are *you* doing today? 

What can I do for you?


In [11]:
import uuid

collection = client.get_or_create_collection("chat-history")

text = f'User: {user_message}\nAssistant: {response}'
texts = [text]
embeddings = embedder.encode(texts).tolist()
ids = [str(uuid.uuid4()) for _ in texts]

collection.add(
    embeddings=embeddings,
    documents=texts,
    ids=ids
)

In [14]:
query_text = 'How are you?'

query_vector = embedder.encode(query_text)

# Query Chroma
results = collection.query(
    query_embeddings=[query_vector],
    n_results=5,
)

print(results)

{'ids': [['9ad8290a-e452-4740-af48-dd12ff68e268']], 'embeddings': None, 'documents': [['User: Hello! how are you?\nAssistant: Hello there! I’m doing great, thank you for asking! As an AI, I don’t really *feel* in the way humans do, but my systems are running smoothly and I’m ready to help you with whatever you need. 😊\n\nHow are *you* doing today? \n\nWhat can I do for you?']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[1.1170940399169922]]}
